In [1]:
import numpy as np
import os
import time

In [2]:
import keras

In [3]:
keras.__version__

'2.4.3'

In [4]:
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [5]:
#!pip install -q -U tensorflow-addons==0.11.2

### Utilities

In [6]:
from utils import verifyDir
from utils.networks import normalize, unnormalize, plot_data

### Dataset

In [7]:
from utils.CIFAR10 import load_real_samples

### Discriminator & Generator

In [8]:
from utils.CIFAR10 import define_discriminator
from utils.CIFAR10 import define_generator

### Semi-Supervised GAN

In [9]:
from utils.networks import define_gan

### Selecting sub-set 

In [10]:
from utils.networks import select_supervised_samples, generate_real_samples
from utils.networks import generate_fake_samples, generate_latent_points

### Training

In [11]:
# train the generator and discriminator
def train(generator_model, unsupervised_model, supervised_model, gan_model, dataset, dataset_test, 
          latent_dim=100, n_epochs=20, n_batch=100, percent_samples=1.0):
    
    # select supervised dataset
    X_sup, y_sup = select_supervised_samples(dataset, percent_samples=percent_samples)
    print("Sup samples:", X_sup.shape, y_sup.shape)
    
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    print('n_epochs=%d, n_batch=%d, batch/epoch=%d, steps=%d' % (n_epochs, n_batch, bat_per_epo, n_steps))
    
    # manually enumerate epochs
    f_history = open(f"{LOG_PATH}SSL_GAN.csv", "w")
    f_history.write("step,generator_loss,unsupervised_real_loss,unsupervised_fake_loss,supervised_loss,supervised_acc,train_loss,test_loss,train_acc,test_acc\n")
    
    #for epoch in n_epochs:
    #    for batch in range(bat_per_epo):
    for step in range(1,n_steps+1):
#         t_start = time.time()
        # update supervised discriminator (c)
        [Xsup_real, ysup_real], _ = generate_real_samples([X_sup, y_sup], n_batch)
        c_loss, c_acc = supervised_model.train_on_batch(Xsup_real, ysup_real)
        
        # update unsupervised discriminator (d)
        [X_real, _], y_real = generate_real_samples(dataset, n_batch)
        d_loss1 = unsupervised_model.train_on_batch(X_real, y_real)
        
        X_fake, y_fake = generate_fake_samples(generator_model, latent_dim, n_batch)
        d_loss2 = unsupervised_model.train_on_batch(X_fake, y_fake)
        
        # update generator (g)
        X_gan, y_gan = generate_latent_points(latent_dim, n_batch), np.ones((n_batch, 1))
        g_loss = gan_model.train_on_batch(X_gan, y_gan)
#         t_total = (time.time() - t_start)
        # summarize loss on this batch
    
        # Train - Test
        X_train, y_train = dataset
        loss_train, acc_train = supervised_model.evaluate(X_train, y_train, verbose=0)

        # evaluate the test classifier model
        X_test, y_test = dataset_test
        loss_test, acc_test = supervised_model.evaluate(X_test, y_test, verbose=0)
        
        # Log
        print('epoch: %d | step: %d | Train: G_Loss: %.3f, ' \
              'D_unsup_loss_real: %.3f, D_unsup_loss_fake: %.3f, ' \
              'D_sup_loss: %.3f, D_sup_acc: %.2f ' \
              'Train acc: %.3f Test acc: %.3f ' %(int(step/bat_per_epo), step, g_loss,
                                                d_loss1, d_loss2,
                                                c_loss, c_acc*100,
                                                 acc_train*100, acc_test*100))#, end = '\r')
        f_history.write(f"{step},{g_loss},{d_loss1},{d_loss2},{c_loss},{c_acc*100},{loss_train},{loss_test},{acc_train*100},{acc_test*100}\n")
        
        if step==1:
            plot_data(unnormalize(X_test).astype(int), 0, "test", grid_size = [10, 10], OUT_PATH=LOG_PATH)
        # evaluate the model performance every so often
        if (step) % (100) == 0 or step == 1:
            #summarize_performance(step, generator_model, supervised_model, latent_dim, dataset, dataset_test)
            # prepare fake examples
            X_generated, _ = generate_fake_samples(generator_model, latent_dim, n_samples=100)
            # scale from [-1,1] to [0,255]
            plot_data(unnormalize(X_generated).astype(int), step, "generated", grid_size = [10, 10], OUT_PATH=LOG_PATH)
            
            X_train, y_train = dataset
            _, acc = supervised_model.evaluate(X_train, y_train, verbose=0)
            print('Train Classifier Accuracy: %.3f%%\n' % (acc * 100))
            
            # evaluate the test classifier model
            X_test, y_test = dataset_test
            _, acc = supervised_model.evaluate(X_test, y_test, verbose=0)
            print('Test Classifier Accuracy: %.3f%%\n' % (acc * 100))
            # save the generator model
            filename2 = f'{LOG_PATH}generator_model_{step}.h5'
            generator_model.save(filename2)
            # save the classifier model
            filename3 = f'{LOG_PATH}supervised_model_{step}.h5'
            supervised_model.save(filename3)
            print('>Saving models Generator: %s and Supervised: %s' % (filename2, filename3))
    
    f_history.close()

### Parameters

In [12]:
input_shape = (32, 32, 3)
num_classes = 10

learning_rate =  0.0002

labeled_rate = 1.0
labeled_samples = 50000*labeled_rate

In [13]:
LOG_PATH = f"Logs/SSGAN_CIFAR10/Classifier_{int(labeled_samples)}/"
verifyDir(LOG_PATH)

### Creating models

In [14]:
# create the discriminator models
unsupervised_model, supervised_model = define_discriminator(in_shape=input_shape, n_classes=num_classes, learning_rate = learning_rate)
# create the generator
generator_model = define_generator(latent_dim=100)

In [15]:
supervised_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 64)        256       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 128)       512       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 128)       0     

In [16]:
unsupervised_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 64)        256       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 128)       512       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 128)       0   

In [17]:
generator_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              413696    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 4096)              0         
_________________________________________________________________
reshape (Reshape)            (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 8, 8, 128)         524416    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 16, 16, 128)       2622

In [18]:
# create the gan
gan_model = define_gan(generator_model, unsupervised_model, learning_rate = learning_rate)

In [19]:
gan_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              413696    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 4096)              0         
_________________________________________________________________
reshape (Reshape)            (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 8, 8, 128)         524416    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 16, 16, 128)       2622

### Loading Dataset

In [20]:
# load image data
dataset, dataset_test = load_real_samples()

(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)


### Training

In [ ]:
train(generator_model, unsupervised_model, supervised_model, gan_model, 
      dataset, dataset_test, latent_dim=100, 
      n_epochs=300, n_batch=128, percent_samples=labeled_rate)

Sup samples: (50000, 32, 32, 3) (50000,)
n_epochs=300, n_batch=128, batch/epoch=390, steps=117000
step: 1 | Train: G_Loss: 0.095, D_unsup_loss_real: 0.080, D_unsup_loss_fake: 2.755, D_sup_loss: 2.710, D_sup_acc: 7.81 Train acc: 14.914 Test acc: 15.230 
Train Classifier Accuracy: 14.914%

Test Classifier Accuracy: 15.230%

>Saving models Generator: Logs/SSGAN_CIFAR10/Classifier_50000/generator_model_1.h5 and Supervised: Logs/SSGAN_CIFAR10/Classifier_50000/supervised_model_1.h5
step: 2 | Train: G_Loss: 0.096, D_unsup_loss_real: 0.089, D_unsup_loss_fake: 1.859, D_sup_loss: 2.298, D_sup_acc: 15.22 Train acc: 18.140 Test acc: 18.320 
step: 3 | Train: G_Loss: 0.097, D_unsup_loss_real: 0.127, D_unsup_loss_fake: 1.205, D_sup_loss: 2.288, D_sup_acc: 18.44 Train acc: 18.934 Test acc: 19.500 
step: 4 | Train: G_Loss: 0.098, D_unsup_loss_real: 0.161, D_unsup_loss_fake: 0.699, D_sup_loss: 2.283, D_sup_acc: 19.55 Train acc: 20.918 Test acc: 21.030 
step: 5 | Train: G_Loss: 0.100, D_unsup_loss_real: 

step: 52 | Train: G_Loss: 0.030, D_unsup_loss_real: 0.047, D_unsup_loss_fake: 0.019, D_sup_loss: 2.206, D_sup_acc: 21.51 Train acc: 15.698 Test acc: 16.060 
step: 53 | Train: G_Loss: 0.019, D_unsup_loss_real: 0.042, D_unsup_loss_fake: 0.021, D_sup_loss: 2.217, D_sup_acc: 16.32 Train acc: 18.368 Test acc: 18.070 
step: 54 | Train: G_Loss: 0.016, D_unsup_loss_real: 0.043, D_unsup_loss_fake: 0.016, D_sup_loss: 2.245, D_sup_acc: 18.37 Train acc: 19.424 Test acc: 19.080 
step: 55 | Train: G_Loss: 0.021, D_unsup_loss_real: 0.041, D_unsup_loss_fake: 0.021, D_sup_loss: 2.241, D_sup_acc: 19.45 Train acc: 19.494 Test acc: 19.420 
step: 56 | Train: G_Loss: 0.017, D_unsup_loss_real: 0.032, D_unsup_loss_fake: 0.012, D_sup_loss: 2.233, D_sup_acc: 19.74 Train acc: 22.624 Test acc: 21.870 
step: 57 | Train: G_Loss: 0.015, D_unsup_loss_real: 0.034, D_unsup_loss_fake: 0.014, D_sup_loss: 2.193, D_sup_acc: 22.17 Train acc: 21.316 Test acc: 21.090 
step: 58 | Train: G_Loss: 0.024, D_unsup_loss_real: 0.031,

step: 103 | Train: G_Loss: 0.001, D_unsup_loss_real: 0.015, D_unsup_loss_fake: 0.003, D_sup_loss: 2.186, D_sup_acc: 23.79 Train acc: 22.428 Test acc: 22.180 
step: 104 | Train: G_Loss: 0.001, D_unsup_loss_real: 0.014, D_unsup_loss_fake: 0.003, D_sup_loss: 2.288, D_sup_acc: 22.47 Train acc: 24.934 Test acc: 24.830 
step: 105 | Train: G_Loss: 0.001, D_unsup_loss_real: 0.012, D_unsup_loss_fake: 0.003, D_sup_loss: 2.212, D_sup_acc: 25.13 Train acc: 21.230 Test acc: 21.020 
step: 106 | Train: G_Loss: 0.001, D_unsup_loss_real: 0.013, D_unsup_loss_fake: 0.003, D_sup_loss: 2.350, D_sup_acc: 21.44 Train acc: 21.010 Test acc: 20.570 
step: 107 | Train: G_Loss: 0.001, D_unsup_loss_real: 0.012, D_unsup_loss_fake: 0.003, D_sup_loss: 2.304, D_sup_acc: 20.97 Train acc: 29.388 Test acc: 28.810 
step: 108 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.012, D_unsup_loss_fake: 0.003, D_sup_loss: 1.983, D_sup_acc: 29.16 Train acc: 20.332 Test acc: 19.980 
step: 109 | Train: G_Loss: 0.000, D_unsup_loss_real:

step: 155 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.008, D_unsup_loss_fake: 0.002, D_sup_loss: 2.737, D_sup_acc: 23.69 Train acc: 30.222 Test acc: 29.150 
step: 156 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.007, D_unsup_loss_fake: 0.002, D_sup_loss: 2.325, D_sup_acc: 29.58 Train acc: 29.534 Test acc: 28.370 
step: 157 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.008, D_unsup_loss_fake: 0.002, D_sup_loss: 2.408, D_sup_acc: 28.70 Train acc: 28.912 Test acc: 27.630 
step: 158 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.008, D_unsup_loss_fake: 0.002, D_sup_loss: 2.351, D_sup_acc: 27.98 Train acc: 32.726 Test acc: 30.870 
step: 159 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.008, D_unsup_loss_fake: 0.002, D_sup_loss: 2.156, D_sup_acc: 31.12 Train acc: 26.824 Test acc: 25.940 
step: 160 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.008, D_unsup_loss_fake: 0.002, D_sup_loss: 2.513, D_sup_acc: 26.29 Train acc: 26.672 Test acc: 26.060 
step: 161 | Train: G_Loss: 0.000, D_unsup_loss_real:

step: 206 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.006, D_unsup_loss_fake: 0.001, D_sup_loss: 2.451, D_sup_acc: 32.01 Train acc: 35.744 Test acc: 34.080 
step: 207 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.006, D_unsup_loss_fake: 0.001, D_sup_loss: 2.258, D_sup_acc: 34.45 Train acc: 36.990 Test acc: 35.430 
step: 208 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.006, D_unsup_loss_fake: 0.001, D_sup_loss: 2.149, D_sup_acc: 35.80 Train acc: 37.092 Test acc: 35.990 
step: 209 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.005, D_unsup_loss_fake: 0.001, D_sup_loss: 2.203, D_sup_acc: 36.27 Train acc: 37.224 Test acc: 35.870 
step: 210 | Train: G_Loss: 0.001, D_unsup_loss_real: 0.005, D_unsup_loss_fake: 0.001, D_sup_loss: 2.127, D_sup_acc: 36.20 Train acc: 31.634 Test acc: 30.220 
step: 211 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.005, D_unsup_loss_fake: 0.001, D_sup_loss: 2.418, D_sup_acc: 30.58 Train acc: 29.308 Test acc: 27.940 
step: 212 | Train: G_Loss: 0.000, D_unsup_loss_real:

...

step: 8959 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.113, D_sup_acc: 43.90 Train acc: 53.134 Test acc: 42.620 
step: 8960 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.263, D_sup_acc: 43.34 Train acc: 53.022 Test acc: 42.580 
step: 8961 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.290, D_sup_acc: 43.29 Train acc: 51.520 Test acc: 41.980 
step: 8962 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.434, D_sup_acc: 42.70 Train acc: 52.098 Test acc: 42.260 
step: 8963 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.361, D_sup_acc: 42.98 Train acc: 51.800 Test acc: 41.830 
step: 8964 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.448, D_sup_acc: 42.56 Train acc: 51.172 Test acc: 41.560 
step: 8965 | Train: G_Loss: 0.000, D_unsup_los

step: 9010 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 5.044, D_sup_acc: 40.83 Train acc: 49.438 Test acc: 40.530 
step: 9011 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.901, D_sup_acc: 41.27 Train acc: 48.378 Test acc: 40.510 
step: 9012 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 5.012, D_sup_acc: 41.25 Train acc: 48.148 Test acc: 40.250 
step: 9013 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 5.099, D_sup_acc: 41.01 Train acc: 48.016 Test acc: 40.160 
step: 9014 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 5.143, D_sup_acc: 40.91 Train acc: 48.964 Test acc: 40.410 
step: 9015 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 5.034, D_sup_acc: 41.14 Train acc: 49.940 Test acc: 40.330 
step: 9016 | Train: G_Loss: 0.000, D_unsup_los

step: 9062 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.734, D_sup_acc: 41.78 Train acc: 49.924 Test acc: 41.090 
step: 9063 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.462, D_sup_acc: 41.81 Train acc: 52.260 Test acc: 42.260 
step: 9064 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.230, D_sup_acc: 42.96 Train acc: 52.428 Test acc: 42.430 
step: 9065 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.314, D_sup_acc: 43.11 Train acc: 52.356 Test acc: 42.650 
step: 9066 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.147, D_sup_acc: 43.36 Train acc: 50.752 Test acc: 41.180 
step: 9067 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.410, D_sup_acc: 41.91 Train acc: 51.264 Test acc: 41.530 
step: 9068 | Train: G_Loss: 0.000, D_unsup_los

step: 9113 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.833, D_sup_acc: 39.27 Train acc: 47.012 Test acc: 38.980 
step: 9114 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.797, D_sup_acc: 39.75 Train acc: 47.228 Test acc: 38.950 
step: 9115 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.750, D_sup_acc: 39.71 Train acc: 48.062 Test acc: 39.570 
step: 9116 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.751, D_sup_acc: 40.33 Train acc: 47.896 Test acc: 39.540 
step: 9117 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.787, D_sup_acc: 40.29 Train acc: 48.058 Test acc: 39.600 
step: 9118 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.742, D_sup_acc: 40.34 Train acc: 49.176 Test acc: 40.740 
step: 9119 | Train: G_Loss: 0.000, D_unsup_los

step: 9165 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.778, D_sup_acc: 40.66 Train acc: 50.178 Test acc: 40.480 
step: 9166 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.655, D_sup_acc: 41.23 Train acc: 50.618 Test acc: 40.690 
step: 9167 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.673, D_sup_acc: 41.42 Train acc: 49.342 Test acc: 39.600 
step: 9168 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.867, D_sup_acc: 40.35 Train acc: 49.742 Test acc: 39.700 
step: 9169 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.854, D_sup_acc: 40.45 Train acc: 50.778 Test acc: 40.810 
step: 9170 | Train: G_Loss: 0.000, D_unsup_loss_real: 0.000, D_unsup_loss_fake: 0.000, D_sup_loss: 4.788, D_sup_acc: 41.56 Train acc: 50.540 Test acc: 40.240 
step: 9171 | Train: G_Loss: 0.000, D_unsup_los

### Testing

In [ ]:
dataset, dataset_test = load_real_samples()

In [ ]:
X_train, y_train = dataset
_, acc = supervised_model.evaluate(X_train, y_train, verbose=0)
print('Train Classifier Accuracy: %.3f%%\n' % (acc * 100))

In [ ]:
X_test, y_test = dataset_test
_, acc = supervised_model.evaluate(X_test, y_test, verbose=0)
print('Test Classifier Accuracy: %.3f%%\n' % (acc * 100))

### Plotting

In [ ]:
import pandas as pd

In [ ]:
results_file = pd.read_csv(f"{LOG_PATH}SSL_GAN.csv")

In [ ]:
log_file = results_file.iloc[:,1:]
log_file

In [ ]:
log_file.iloc[:, [0,1,2,3]].plot(figsize=(12,8))

In [ ]:
log_file.iloc[:, [5,6]].plot(figsize=(12,8))

In [ ]:
log_file.iloc[:, [7,8]].plot(figsize=(12,8), ylim=(0,100), yticks=range(0,110,10))